# Deep Learning - Reconhecimento de Dígitos

Este notebook realiza uma classificação de imagens utilizando Redes Neurais Profundas (Deep Learning), sob os dados da competição [Digit Recognizer](https://www.kaggle.com/c/digit-recognizer), também conhecido como MNIST. Esta competição tem como propósito identificar qual o número desenhado numa imagem. Sendo assim, nesta competição vamos praticar conceitos de Visão Computacional (Computer Vision) para classificar as imagens corretamente.

> Conteúdo voltado para nível intermediário de Aprendizado de Máquina e Ciência de Dados!

<a id="top"></a>

## Conteúdo

> **Nota.** Alguns códigos foram ocultados para dar destaque aos conteúdos mais importantes.

O notebook está organizado como segue:

- [Classificação](#clas) - Deep Learning
- [Predição](#pred) - Geração do arquivo de submissão.

In [1]:
!pip install torchtext

In [1]:
!pip install fastai==2.2.5

# Visualização dos Dados

In [1]:
import os
import pandas as pd


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Leitura dos conjuntos de dados, e adaptando as categorias `label` para `str`.

In [1]:
# conjunto de teste
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

# conjunto de treino
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_train = df_train.astype({'label': 'category'})

In [1]:
df_train.sample(3)

In [1]:
import matplotlib.pyplot as plt

example = df_train.drop(labels=['label'], axis=1).iloc[9]
example = example.values.reshape(-1,28,28)
plt.imshow(example[0,:,:], cmap="gray")

<a id="clas"></a>

# Classificação

Construção da arquitetura de Deep Learning, e treinamento do modelo.

[Voltar para o Topo](#top)

In [1]:
from fastai.tabular.all import *

## Conjunto de Treinamento e Validação

Separa o conjunto de treinamento em treinamento e validação (20% do conjunto).

In [1]:
# Transformations
dls = TabularDataLoaders.from_df(df_train, y_names='label', seed=26, valid_pct=0.2)

## Arquitetura

Criamos uma Rede Neural com 4 camadas escondidas, sendo elas `[100, 200, 200, 100]`.

In [1]:
learn = tabular_learner(dls, layers=[100, 200, 200, 100], metrics=accuracy)

Treinamento, ou seja, o aprendizado.

In [1]:
learn.fit(3)

Verificando o `loss` do conjunto de treinamento e validação.

In [1]:
learn.recorder.plot_loss()

<a id="pred"></a>
    
# Predição

Vamos predizer as classes do conjunto de teste, e criar um arquivo de submissão.

[Voltar para o Topo](#top)


Testando a predição para um elemento.

In [1]:
_, clas, probs = learn.predict(df_test.iloc[0])

In [1]:
print(f'Classe {clas}, probabilidade {probs[clas]}')

Todas as classes

In [1]:
learn.dls.vocab

Predizendo todo o conjunto de teste.

In [1]:
# conjunto de teste
dl = learn.dls.test_dl(df_test)

In [1]:
# pega as probabilidades
probs, _ = learn.get_preds(dl=dl)

In [1]:
# pega aquela classe de maior probabilidade
y_pred = torch.argmax(probs,dim=1)
y_pred = y_pred.numpy()
display(y_pred)

Gera o arquivo de submissão.

In [1]:
# Aumenta o index por 1, para se adequar ao arquivo de submissão
df_test.index += 1

In [1]:
submission = {'ImageId':df_test.index, 'Label':y_pred}
df_submission = pd.DataFrame(submission)

In [1]:
df_submission.sample(5)

In [1]:
# gerando arquivo de submissão
df_submission.to_csv('submission.csv', index=False)